In [14]:
import pandas as pd
import numpy as np
import re


In [16]:
dataset_path = 'Datasets/IndianRecipeData.xlsx'

indian_recipe_data = pd.read_excel(dataset_path)

print(indian_recipe_data.head())


   Srno                                         RecipeName  \
0     1                               Masala Karela Recipe   
1     2  टमाटर पुलियोगरे रेसिपी - Spicy Tomato Rice (Re...   
2     3  Ragi Semiya Upma Recipe - Ragi Millet Vermicel...   
3     4  Gongura Chicken Curry Recipe - Andhra Style Go...   
4     5  आंध्रा स्टाइल आलम पचड़ी रेसिपी - Adrak Chutney ...   

                                TranslatedRecipeName  \
0                               Masala Karela Recipe   
1                         Spicy Tomato Rice (Recipe)   
2  Ragi Semiya Upma Recipe - Ragi Millet Vermicel...   
3  Gongura Chicken Curry Recipe - Andhra Style Go...   
4  Andhra Style Alam Pachadi Recipe - Adrak Chutn...   

                                         Ingredients  \
0  6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...   
1  2-1/2 कप चावल - पका ले,3 टमाटर,3 छोटा चमच्च बी...   
2  1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...   
3  500 grams Chicken,2 Onion - chopped,1 Tomato -...   
4  1 बड़ा च

In [17]:
# Drop duplicates if any
indian_recipe_data.drop_duplicates(inplace=True)

# Handle missing values if needed
indian_recipe_data.dropna(inplace=True)



In [18]:
column_names = indian_recipe_data.columns

# Print the column names
print(column_names)

Index(['Srno', 'RecipeName', 'TranslatedRecipeName', 'Ingredients',
       'TranslatedIngredients', 'PrepTimeInMins', 'CookTimeInMins',
       'TotalTimeInMins', 'Servings', 'Cuisine', 'Course', 'Diet',
       'Instructions', 'TranslatedInstructions', 'URL'],
      dtype='object')


In [104]:
import pandas as pd

# Load your dataset
dataset_path = 'Datasets/IndianRecipeData.xlsx'
indian_recipe_data = pd.read_excel(dataset_path)

# Check if the "Course" column exists in the dataset
if 'Course' in indian_recipe_data.columns:
    # Get the unique entries in the "Course" column
    course_labels = indian_recipe_data['Course'].unique()
    
    # Print the unique course labels
    print("Unique Course Labels:")
    for label in course_labels:
        print(label)
else:
    print("The dataset does not contain a 'Course' column.")

Unique Course Labels:
Side Dish
Main Course
South Indian Breakfast
Lunch
Snack
High Protein Vegetarian
Dinner
Appetizer
Indian Breakfast
Dessert
North Indian Breakfast
One Pot Dish
World Breakfast
Non Vegeterian
Vegetarian
Eggetarian
No Onion No Garlic (Sattvic)
Brunch
Vegan
Sugar Free Diet


In [106]:
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Define your dataset path
dataset_path = 'Datasets/IndianRecipeData.xlsx'
indian_recipe_data = pd.read_excel(dataset_path)


#------------------------- Define sample user input ingredients-------------------------------------
sample_user_input = "ragi"
#---------------------------------------------------------------------------------------------------

# ------------------------Define sample user input course and time---------------------------------------
sample_course_input = "satvic"
sample_max_time_input = 30  # in minutes
#----------------------------------------------------------------------------------------------------

def preprocess_text(text):
    if isinstance(text, str):  # Check if input is a string
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
        text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
        return text.strip()
    else:
        return ''  # Return empty string for non-string inputs

def calculate_similarity(user_ingredients, recipe_ingredients, vectorizer):
    user_vector = vectorizer.transform([user_ingredients])
    recipe_vector = vectorizer.transform([recipe_ingredients])
    similarity_score = cosine_similarity(user_vector, recipe_vector)[0][0]
    return similarity_score

def recommend_recipes(user_ingredients, recipe_data, vectorizer):
    priority_recipes = []
    other_recipes = []

    # Split the user input ingredients into a list
    user_ingredients_list = user_ingredients.split(", ")
    
    for index, row in recipe_data.iterrows():
        recipe_ingredients = preprocess_text(row['Ingredients'])
        similarity_score_name = calculate_similarity(user_ingredients, preprocess_text(row['RecipeName']), vectorizer)
        similarity_score_translated = calculate_similarity(user_ingredients, preprocess_text(row['TranslatedRecipeName']), vectorizer)
        similarity_score_ingredients = calculate_similarity(user_ingredients, recipe_ingredients, vectorizer)

        # Check if all user ingredients are in the recipe ingredients
        if all(ingredient.strip() in recipe_ingredients.split(", ") for ingredient in user_ingredients_list):
            priority_recipes.append((row['RecipeName'], row['Ingredients'], row['Instructions'], 1.0))  # Set similarity score to 1 for exact matches
        # Check if the entered ingredient is in the recipe name as a major element
        elif similarity_score_name > 0.5:
            priority_recipes.append((row['RecipeName'], row['Ingredients'], row['Instructions'], similarity_score_name))
        # Check if the entered ingredient is in the translated recipe name as a major element
        elif similarity_score_translated > 0.5:
            priority_recipes.append((row['RecipeName'], row['Ingredients'], row['Instructions'], similarity_score_translated))
        # Check if the entered ingredient is in the recipe ingredients
        elif similarity_score_ingredients > 0.5:
            priority_recipes.append((row['RecipeName'], row['Ingredients'], row['Instructions'], similarity_score_ingredients))
        else:
            other_recipes.append((row['RecipeName'], row['Ingredients'], row['Instructions'], 0))  # Set a low similarity score for others

    priority_recipes.sort(key=lambda x: x[3], reverse=True)
    other_recipes.sort(key=lambda x: x[3], reverse=True)

    return priority_recipes, other_recipes



def generate_sequence_of_recommendations(priority_recipes, other_recipes):
    sequence_recommendations = []
    item_number = 1

    for recipe, _, _, _ in priority_recipes:
        sequence_recommendations.append(f"{item_number}. {recipe}")
        item_number += 1

    for recipe, _, _, _ in other_recipes:
        sequence_recommendations.append(f"{item_number}. {recipe}")
        item_number += 1

    return sequence_recommendations

def recommendation_system(user_input, course_input, max_time_input, dataset):
    user_input_processed = preprocess_text(user_input)
    dataset['Ingredients'] = dataset['Ingredients'].apply(preprocess_text)

    vectorizer = TfidfVectorizer()
    vectorizer.fit(dataset['Ingredients'])

    priority_recipes, other_recipes = recommend_recipes(user_input_processed, dataset, vectorizer)
    sequence_recommendations = generate_sequence_of_recommendations(priority_recipes, other_recipes)

    return sequence_recommendations, priority_recipes

# Call the recommendation_system function with the sample input and your dataset
sequence_recommendations, priority_recipes = recommendation_system(sample_user_input, sample_course_input, sample_max_time_input, indian_recipe_data)

# Print the recommended recipes
print("Recommended Recipes:")
for item in sequence_recommendations:
    print(item)


Recommended Recipes:
1. Ragi Sankati Recipe - Andhra Style Ragi Mudda
2. Ragi Halwa Recipe | Healthy Indian Sweet Made With Ragi
3. Ragi Nippattu Recipe With Peanuts And Sesame - Ragi Thattai / Chekkalu
4. Ragi Wheat Phulka Recipe- Diabetic Friendly Ragi Roti
5. Ragi Semiya Upma Recipe - Ragi Millet Vermicelli Breakfast
6. Soft & Delicious Ragi Mudde Recipe - Karnataka Style Ragi Balls
7. Healthy Ragi & Rice Breakfast Porridge Recipe
8. Ragi Halbai Recipe - Karnataka Style Ragi Halwa Recipe
9. Ragi Or Finger Millet Powder/ Malt Recipe for Babies 6 Months & Above
10. रागी कांजी रेसिपी - Ragi Kanji (Recipe In Hindi)
11. Sweet Ragi Malt Recipe - Healthy Ragi Kanji/ Porridge Recipe
12. Ragi Murukku And Kodubale Recipe -Chakli And Kodubale
13. Ragi Masala Twisters Recipe-Healthy Snack
14. Masala Karela Recipe
15. टमाटर पुलियोगरे रेसिपी - Spicy Tomato Rice (Recipe In Hindi)
16. Gongura Chicken Curry Recipe - Andhra Style Gongura Chicken
17. आंध्रा स्टाइल आलम पचड़ी रेसिपी - Adrak Chutney (Reci

In [86]:
from IPython.display import display, Markdown

# Hardcoded value representing the item number to print the detailed recipe
selected_item_number = 8  # Change this to the desired item number

# Check if the selected item number is valid
if 1 <= selected_item_number <= len(sequence_recommendations):
    selected_item_str = sequence_recommendations[selected_item_number - 1]  # Get the item string
    selected_item_name = selected_item_str.split(". ")[1]  # Extract recipe name

    # Find the recipe details in the dataset
    selected_recipe_details = indian_recipe_data[indian_recipe_data['RecipeName'] == selected_item_name]

    if not selected_recipe_details.empty:
        selected_recipe_details = selected_recipe_details.iloc[0]  # Take the first row (assuming unique recipe names)
        display(Markdown("### Recipe for Selected Item:"))
        display(Markdown(f"**Recipe Name:** {selected_recipe_details['RecipeName']}"))
        display(Markdown(f"**Ingredients:**\n{selected_recipe_details['Ingredients']}"))
        display(Markdown(f"**Instructions:**\n{selected_recipe_details['Instructions']}"))
    else:
        print("Recipe details not found.")
else:
    print("Invalid item number. Please enter a valid number.")


### Recipe for Selected Item:

**Recipe Name:** Easy Homemade Sugar Free Dates Flavored Yogurt Recipe

**Ingredients:**
500 ml milk boiled and cooled2 teaspoon milk powder12 cup dates pitted2 teaspoons curd dahi yogurt

**Instructions:**
To begin making Easy Homemade Sugar Free Dates Flavored Yogurt Recipe, blend dates with just enough milk to make a fine paste.Run the date paste through a sieve with small holes to remove all the skin. Keep the smooth date paste aside.Heat the rest of the milk in a saucepan; stir in powdered milk and mix well.Reduce heat and stir in the date paste; whisk until well incorporated and smooth. Heat the whole mixture until it is hot but not boiling.Remove from heat and let cool until it is lukewarm .you should be able to hold a clean finger in it for at least 10 seconds without feeling uncomfortable. Do not let it cool too much.Once it reaches that temperature, gently stir in the starter culture; plain yogurt and whisk until smooth.Gently decant the date flavored mixture into containers in which you want the yogurt to set.There are various ways you can incubate the mixture to form yogurt. I normally place the bowls in an insulated container which has a lid and leave it aside for 6 to 8 hours or until the yogurt has set completely; it all depends upon the culture used, the temperature of the yogurt, and whether you want it sweet or sour.If you have an oven, you can turn the oven on and heat it until it reaches 120 degrees F; turn it off, place the yogurt container in the oven, close the door but leave the oven light on.Incubate for 4 to 6 hours or more if you want a tarter yogurt. Check in between to see if it has set.Once set, refrigerate until serving time. Enjoy sugar free date yogurt topped with more dates!Serve Easy Homemade Sugar Free Dates Flavored Yogurt Recipe as a cheat dessert after a dinner like Paneer Pulao With Green Peas Recipe, Yellow Pumpkin Dal Recipe and Kachumber Salad Recipe With Cucumber, Onion & Tomatoes.
